In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/favorita/'

In [3]:
!ls {PATH}

holidays_events.csv  oil.csv		    stores.csv	train.csv
items.csv	     sample_submission.csv  test.csv	transactions.csv


In [4]:
# table_names = ['train', 'stores', 'transactions', 'items', 'oil', 'holidays_events', 'test']

In [4]:
dtypes = {'store_nbr': np.dtype('int64'),
          'item_nbr': np.dtype('int64'),
          'unit_sales': np.dtype('float64'),
          'onpromotion': np.dtype('O')}

In [5]:
train = pd.read_csv(f'{PATH}train.csv', index_col='id', parse_dates=['date'], dtype=dtypes)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
len(train)

125497040

In [ ]:
# Master list of Items
unique_items = list(set(train['item_nbr'].unique())) # | set(test['item_nbr'].unique()))
unique_items.sort()
num_unique_items = len(unique_items)

# Master list of stores
unique_stores = list(set(train['store_nbr'].unique())) # | set(test['Store'].unique()))
unique_stores.sort()
num_unique_stores = len(unique_stores)

In [6]:
# tables = [pd.read_csv(f'{PATH}{fname}.csv', dtype=dtypes) for fname in table_names]

In [7]:
from IPython.display import HTML

In [8]:
for i in range(0, len(table_names)): display(tables[i].head(), table_names[i])
# for t in tables: display(t.head()), display(table_names[t.index])

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


'train'

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


'stores'

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


...

'oil'

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


'holidays_events'

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


'test'

In [ ]:
for t in tables: display(DataFrameSummary(t).summary())

In [ ]:
train, test, stores, transactions, oil, items, holidays_events = tables

In [ ]:
len(train),len(test)

In [ ]:
add_datepart(train, "date", drop=False)
add_datepart(test, "date", drop=False)
add_datepart(transactions, "date", drop=False)
add_datepart(oil, "date", drop=False)
add_datepart(holidays_events, "date", drop=False)

In [ ]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [ ]:
joined = join_df(train, items, "item_nbr")
joined_test = join_df(test, items, "item_nbr")
len(joined[joined.item_nbr.isnull()]),len(joined_test[joined_test.item_nbr.isnull()])